### Làm sạch tập dữ liệu sau khi crawl (mùa giải 2000-2001 -> 2022-2023)

In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [2]:
match_df = pd.read_csv('../raw_data/raw_data_train.csv')
match_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,attendance,captain,formation,referee,match report,season,team
0,8/7/2022,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,62443.0,İlkay Gündoğan,4/3/2003,Michael Oliver,Match Report,2022-2023,Manchester City
1,8/13/2022,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,53453.0,İlkay Gündoğan,4-2-3-1,David Coote,Match Report,2022-2023,Manchester City
2,8/21/2022,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,52258.0,İlkay Gündoğan,4/3/2003,Jarred Gillett,Match Report,2022-2023,Manchester City
3,8/27/2022,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,53112.0,Kevin De Bruyne,4-2-3-1,Darren England,Match Report,2022-2023,Manchester City
4,8/31/2022,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,53409.0,İlkay Gündoğan,4-2-3-1,Paul Tierney,Match Report,2022-2023,Manchester City


In [3]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          17480 non-null  object 
 1   time          6840 non-null   object 
 2   comp          17480 non-null  object 
 3   round         17480 non-null  object 
 4   day           17480 non-null  object 
 5   venue         17480 non-null  object 
 6   result        17480 non-null  object 
 7   gf            17480 non-null  int64  
 8   ga            17480 non-null  int64  
 9   opponent      17480 non-null  object 
 10  attendance    15560 non-null  float64
 11  captain       6080 non-null   object 
 12  formation     11410 non-null  object 
 13  referee       17480 non-null  object 
 14  match report  17480 non-null  object 
 15  season        17480 non-null  object 
 16  team          17480 non-null  object 
dtypes: float64(1), int64(2), object(14)
memory usage: 2.3+ MB


### Clean data

1. Xóa các features không dùng cho việc build model

In [4]:
match_df = match_df.drop(columns=["comp", "day", "match report", "captain"])

match_df.columns

Index(['date', 'time', 'round', 'venue', 'result', 'gf', 'ga', 'opponent',
       'attendance', 'formation', 'referee', 'season', 'team'],
      dtype='object')

2. Xử lý các giá trị NaN

In [5]:
match_df = match_df.dropna(subset=["result"])
match_df.shape

(17480, 13)

3. Xử lý formation bị lỗi dữ liệu

In [6]:
match_df['formation'].value_counts()

formation
4-2-3-1       3081
4/4/2002      2353
4/3/2003      1676
4-4-1-1       1040
4/5/2001       710
3/4/2003       690
4-1-4-1        614
3/5/2002       311
4-1-2-1-2◆     191
5/4/2001       148
5/3/2002       137
3-4-1-2        124
3-5-1-1         78
4-3-1-2         72
4-2-2-2         68
4-3-2-1         44
3-1-4-2         38
3-3-3-1         12
3-2-4-1         12
4-1-3-2          6
3-4-3◆           4
4-2-4-0          1
Name: count, dtype: int64

In [7]:
choices = ["4-2-3-1", "4-4-2", "4-3-3"]

# Fill NaN values with randomly chosen values from the list
match_df["formation"] = match_df["formation"].fillna(np.random.choice(choices))

match_df["formation"].value_counts()

formation
4-3-3         6070
4-2-3-1       3081
4/4/2002      2353
4/3/2003      1676
4-4-1-1       1040
4/5/2001       710
3/4/2003       690
4-1-4-1        614
3/5/2002       311
4-1-2-1-2◆     191
5/4/2001       148
5/3/2002       137
3-4-1-2        124
3-5-1-1         78
4-3-1-2         72
4-2-2-2         68
4-3-2-1         44
3-1-4-2         38
3-3-3-1         12
3-2-4-1         12
4-1-3-2          6
3-4-3◆           4
4-2-4-0          1
Name: count, dtype: int64

In [8]:
# Xóa kí tự đặc biệt trong data cột 'formation'
match_df['formation'] = match_df['formation'].str.replace('◆','', regex=False)

def nomalize_formation(value):
    if "/" in value:
        parts = value.split("/")
        parts[2] = parts[2][-1]
        return "-".join(parts)
    elif value.endswith('-0'):
        return value[:-2]
    else:
        return value

match_df['formation'] = match_df['formation'].apply(nomalize_formation)
match_df['formation'].unique()

array(['4-3-3', '4-2-3-1', '3-4-3', '3-2-4-1', '4-4-2', '4-1-4-1',
       '4-1-2-1-2', '3-3-3-1', '3-4-1-2', '3-5-1-1', '3-5-2', '4-3-1-2',
       '4-3-2-1', '4-2-2-2', '5-3-2', '5-4-1', '3-1-4-2', '4-4-1-1',
       '4-5-1', '4-1-3-2', '4-2-4'], dtype=object)

4. Xử lý attendance bị missing data

In [9]:
# Fill giá trị thiếu trong cột 'attendance' bằng cách dùng group theo 2 đội đối đầu nhau rồi lấy trung bình
# vì tùy cặp đấu mà lượng khán giả sẽ cao
match_df['attendance'] = match_df.groupby(['team', 'opponent'])['attendance'].transform(lambda x: x.fillna(x.mean()))

# Vì có thể có các cặp chưa từng đấu nên sẽ còn giá trị thiếu => ta sẽ tiếp tục fill data theo trung bình đội đó
match_df['attendance'] = match_df.groupby('team')['attendance'].transform(lambda x: x.fillna(x.mean()))
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        17480 non-null  object 
 1   time        6840 non-null   object 
 2   round       17480 non-null  object 
 3   venue       17480 non-null  object 
 4   result      17480 non-null  object 
 5   gf          17480 non-null  int64  
 6   ga          17480 non-null  int64  
 7   opponent    17480 non-null  object 
 8   attendance  17480 non-null  float64
 9   formation   17480 non-null  object 
 10  referee     17480 non-null  object 
 11  season      17480 non-null  object 
 12  team        17480 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 1.7+ MB


5. Điền giá trị mặc định cho cột time

In [10]:
match_df["time"] = match_df["time"].fillna("00:00")
match_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        17480 non-null  object 
 1   time        17480 non-null  object 
 2   round       17480 non-null  object 
 3   venue       17480 non-null  object 
 4   result      17480 non-null  object 
 5   gf          17480 non-null  int64  
 6   ga          17480 non-null  int64  
 7   opponent    17480 non-null  object 
 8   attendance  17480 non-null  float64
 9   formation   17480 non-null  object 
 10  referee     17480 non-null  object 
 11  season      17480 non-null  object 
 12  team        17480 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 1.7+ MB


6. Sửa lại tên đội đúng

In [11]:
# Hàm lấy ra dataFrame tên khác nhau của 2 cột
def compare_team_component():
    sorted_unique_teams = sorted(match_df["team"].unique())
    sorted_unique_opponents = sorted(match_df["opponent"].unique())

    # Tạo DataFrame để hiển thị các tên đội đã được sắp xếp
    team_df = pd.DataFrame(sorted_unique_teams, columns=["Team Names"])
    opponent_df = pd.DataFrame(sorted_unique_opponents, columns=["Opponent Names"])

    # Vì số lượng tên trong mỗi danh sách có thể không bằng nhau, ta cần thêm các hàng trống cho DataFrame ngắn hơn
    max_length = max(len(team_df), len(opponent_df))
    team_df = team_df.reindex(range(max_length)).fillna("")
    opponent_df = opponent_df.reindex(range(max_length)).fillna("")

    # Kết hợp hai DataFrame để dễ so sánh
    comparison_df = pd.concat([team_df, opponent_df], axis=1)
    comparison_df = comparison_df[
        comparison_df["Team Names"] != comparison_df["Opponent Names"]
    ]
    return comparison_df


comparison_df = compare_team_component()
comparison_df

,Team Names,Opponent Names
3,Blackburn Rovers,Blackburn
5,Bolton Wanderers,Bolton
9,Brighton and Hove Albion,Brighton
12,Charlton Athletic,Charlton Ath
19,Huddersfield Town,Huddersfield
26,Manchester United,Manchester Utd
28,Newcastle United,Newcastle Utd
30,Nottingham Forest,Nott'ham Forest
32,Queens Park Rangers,QPR
34,Sheffield United,Sheffield Utd


In [12]:
# Mảng các tên đội cần đổi lại để đồng nhất 2 cột
rename_dict = {
    "Brighton": "Brighton and Hove Albion",
    "Manchester Utd": "Manchester United",
    "Newcastle Utd": "Newcastle United",
    "Nott'ham Forest": "Nottingham Forest",
    "Sheffield Utd": "Sheffield United",
    "Tottenham": "Tottenham Hotspur",
    "West Brom": "West Bromwich Albion",
    "West Ham": "West Ham United",
    "Wolves": "Wolverhampton Wanderers",
    "Blackburn": "Blackburn Rovers",
    "Bolton": "Bolton Wanderers",
    "Charlton Ath": "Charlton Athletic",
    "Huddersfield": "Huddersfield Town",
    "QPR": "Queens Park Rangers",
}

match_df['opponent'] = match_df['opponent'].replace(rename_dict)

comparison_df = compare_team_component()
comparison_df

,Team Names,Opponent Names


### Check lại data lần cuối

In [13]:
pd.to_datetime(match_df["date"]).dt.year.value_counts().sort_index()

date
2000    410
2001    746
2002    782
2003    718
2004    784
2005    748
2006    788
2007    742
2008    758
2009    756
2010    748
2011    754
2012    782
2013    744
2014    760
2015    760
2016    756
2017    802
2018    742
2019    758
2020    672
2021    816
2022    722
2023    432
Name: count, dtype: int64

In [14]:
match_df["season"].value_counts()

season
2022-2023    760
2010-2011    760
2001-2002    760
2002-2003    760
2003-2004    760
2004-2005    760
2005-2006    760
2006-2007    760
2007-2008    760
2008-2009    760
2009-2010    760
2011-2012    760
2021-2022    760
2012-2013    760
2013-2014    760
2014-2015    760
2015-2016    760
2016-2017    760
2017-2018    760
2018-2019    760
2019-2020    760
2020-2021    760
2000-2001    760
Name: count, dtype: int64